In [1]:
import os
os.environ["WANDB_MODE"] = "offline"


In [2]:
import torch

torch.cuda.empty_cache()
# Imports
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification,TrainerCallback
import torch
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from datasets import load_dataset, Dataset
import scipy.special
import os
import json
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Define the working device


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels= 2
# Load the model
model = AutoModelForSequenceClassification.from_pretrained("chenyuhe/CaVepP53", trust_remote_code=True)
tokenizer=model.tokenizer
model = model.to(device)

In [3]:
for param in model.parameters():
    param.requires_grad = False
# Unfreeze the last n layers
# Here, we assume the model has a known number of layers.
# You may need to adjust this based on your specific model architecture.
last_n_layers = 22
for name, param in model.named_parameters():
    # Compute the index of the current layer
    layer_index = len(list(model.named_parameters())) - list(model.named_parameters()).index((name, param)) - 1
    # If the layer index is within the last n layers, unfreeze it
    if layer_index < last_n_layers:
        param.requires_grad = True
        print(name)

transformer.blocks.35.attn.layernorm_qkv.0.weight
transformer.blocks.35.attn.layernorm_qkv.0.bias
transformer.blocks.35.attn.layernorm_qkv.1.weight
transformer.blocks.35.attn.out_proj.weight
transformer.blocks.35.attn.q_ln.weight
transformer.blocks.35.attn.k_ln.weight
transformer.blocks.35.ffn.0.weight
transformer.blocks.35.ffn.0.bias
transformer.blocks.35.ffn.1.weight
transformer.blocks.35.ffn.3.weight
transformer.norm.weight
sequence_head.0.weight
sequence_head.0.bias
sequence_head.2.weight
sequence_head.2.bias
sequence_head.3.bias
classifier.0.weight
classifier.0.bias
classifier.2.weight
classifier.2.bias
classifier.3.weight
classifier.3.bias


In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
from tqdm import tqdm
from Bio import SeqIO

# Load the promoter dataset from the InstaDeep Hugging Face resources
# Note: By default, load_dataset("csv") will treat the entire file as a single split (named "train").
data_file = "./datas/train_dataset.csv"  # <-- your path

# After loading the data, we get a DatasetDict containing a single split (i.e., dataset["train"]).
raw_data = load_dataset("csv", data_files=data_file, encoding="utf-8")
all_data = raw_data["train"]
# Load the WT sequence
TP53_pro = SeqIO.read(r"./datas/p53_protein_sequence.fasta", "fasta")
WT_seq = str(TP53_pro.seq)

# Define a function to generate mutant sequences
def generate_mut_sequence(row,WT_seq=WT_seq):
    pos = row["POS"]
    prof = row['REF']
    prol = row['ALT']
    WT_seq = WT_seq  # Ensure WT_seq is used correctly

    # Check if position is out of range or REF does not match
    if pos - 1 >= len(WT_seq) or WT_seq[pos - 1] != prof:
        print(f"Error at index {row['id']}: POS out of range or REF mismatch")  # Print error message
        print(row)
        return {"seq": None}  # Return None for invalid sequences

    # Generate mutant sequence
    #
    left_start = max(0, pos - 26)
    right_end = min(len(WT_seq), pos + 25)
    left_pad = "<pad>" * (26 - (pos - left_start))
    right_pad = "<pad>" * (25 - (right_end - pos + 1))

    mutseq = left_pad + WT_seq[left_start:pos - 1] + prol + WT_seq[pos:right_end] + right_pad
    return {"seq": mutseq}

# Apply the function to each row in the dataset
all_data = all_data.map(generate_mut_sequence, batched=False)

# Filter out rows with invalid sequences
all_data = all_data.filter(lambda row: row['seq'] is not None)

print("Total samples in all_data:", len(all_data))


# Split the data into training, validation, and test sets
train_val_test = all_data.train_test_split(test_size=0.2, seed=42)  # Split into train and temp (20%)
train_dataset_raw = train_val_test["train"]
val_test_data = train_val_test["test"]

val_test_split = val_test_data.train_test_split(test_size=0.5, seed=42)  # Split temp into val and test (50% each)
validation_dataset_raw = val_test_split["train"]
test_dataset_raw = val_test_split["test"]

print("Train set:", len(train_dataset_raw))
print("Validation set:", len(validation_dataset_raw))
print("Test set:", len(test_dataset_raw))

# Load the tokenizer
tokenizer = model.tokenizer

def tokenize_function(examples):
    outputs = tokenizer(examples["seq"], padding=True, return_tensors='pt')
    return outputs

# Creating tokenized promoter dataset
tokenized_datasets_train_promoter = train_dataset_raw.map(
    tokenize_function,
    batched=True,
    remove_columns=["seq", "Name", "REF", "ALT","POS"],
)
tokenized_datasets_validation_promoter = validation_dataset_raw.map(
    tokenize_function,
    batched=True,
    remove_columns=["seq", "Name", "REF", "ALT","POS"],
)
tokenized_datasets_test_promoter = test_dataset_raw.map(
    tokenize_function,
    batched=True,
    remove_columns=["seq", "Name", "REF", "ALT","POS"],
)



Total samples in all_data: 3522
Train set: 2817
Validation set: 352
Test set: 353


Map:   0%|          | 0/2817 [00:00<?, ? examples/s]

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

Map:   0%|          | 0/353 [00:00<?, ? examples/s]

In [ ]:
batch_size = 64
model_name='CaVepP53'
args_promoter = TrainingArguments(
    f"{model_name}-finetuned-TP53_51AA",
    remove_unused_columns=False,
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps= 1,
    per_device_eval_batch_size= 64,
    num_train_epochs= 2,
    logging_steps= 50,
    weight_decay=4e-5,
    save_steps = 50,
    load_best_model_at_end=True,  # Keep the best model according to the evaluation
    metric_for_best_model="f1_score",
    label_names=["labels"],
    dataloader_drop_last=True,
    max_steps= 1000,
    seed=42,
    save_safetensors=False
)

# Define the metric for the evaluation using the f1 score
def compute_metrics_f1_score(eval_pred):
    """Computes F1 score for binary classification"""
    logits = eval_pred.predictions[0]
    predictions = np.argmax(logits, axis=-1)
    references = eval_pred.label_ids
    probs = scipy.special.softmax(logits, axis=-1)[:, 1]
    r={'f1_score': f1_score(references, predictions),
       "precision": precision_score(references, predictions),
       "recall":  recall_score(references, predictions),
       "roc_auc": roc_auc_score(references, probs)}
    return r

# A custom callback for logging training/validation metrics and writing them to a JSON file.
class MetricsLoggerCallback(TrainerCallback):
    def __init__(self, json_file_path="training_metrics.json"):
        super().__init__()
        self.json_file_path = json_file_path
        self.metrics = {
            "train_loss": [],
            "eval_loss": [],
            "f1_score": [],
            "precision": [],
            "recall": [],
            "roc_auc": []
        }
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        """
        Each time the Trainer logs information (i.e., at logging_steps), this section is executed.
        The logs contain information such as "loss", "learning_rate", "epoch", etc.
        """
        if logs is None:
            return
        
        # train_loss
        if "loss" in logs:
            self.metrics["train_loss"].append({
                "step": state.global_step,
                "value": logs["loss"]
            })
        
        # save json
        with open(self.json_file_path, "w") as f:
            json.dump(self.metrics, f, indent=4)

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """
        Each time validation is performed (when evaluation_strategy="steps", 
        it triggers evaluation every specified number of steps), the execution occurs here.
        The metrics contain indicators such as eval_loss, eval_f1_score, eval_precision, and so on.
        """
        if metrics is None:
            return
        
        # eval_loss
        if "eval_loss" in metrics:
            self.metrics["eval_loss"].append({
                "step": state.global_step,
                "value": metrics["eval_loss"]
            })
        # f1
        if "eval_f1_score" in metrics:
            self.metrics["f1_score"].append({
                "step": state.global_step,
                "value": metrics["eval_f1_score"]
            })
        # precision
        if "eval_precision" in metrics:
            self.metrics["precision"].append({
                "step": state.global_step,
                "value": metrics["eval_precision"]
            })
        # recall
        if "eval_recall" in metrics:
            self.metrics["recall"].append({
                "step": state.global_step,
                "value": metrics["eval_recall"]
            })
        # roc_auc
        if "eval_roc_auc" in metrics:
            self.metrics["roc_auc"].append({
                "step": state.global_step,
                "value": metrics["eval_roc_auc"]
            })
        
        # save in json
        with open(self.json_file_path, "w") as f:
            json.dump(self.metrics, f, indent=4)

            
trainer = Trainer(
    model=model.to(device),
    args=args_promoter,
    train_dataset=tokenized_datasets_train_promoter,
    eval_dataset=tokenized_datasets_validation_promoter,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_f1_score,
    callbacks=[MetricsLoggerCallback("training_metrics.json")]  
)

train_results = trainer.train()

/home/huang/anaconda3/envs/ESM/lib/python3.13/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_674057/273578501.py:116: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,F1 Score,Precision,Recall,Roc Auc
50,0.283800,0.206810,0.918768,0.906077,0.931818,0.974511
100,0.235500,0.252604,0.900302,0.961290,0.846591,0.975221
150,0.230400,0.211085,0.919540,0.930233,0.909091,0.973090
200,0.212600,0.307105,0.901857,0.845771,0.965909,0.962831
250,0.241900,0.319584,0.866242,0.985507,0.772727,0.970841
300,0.202800,0.236346,0.902507,0.885246,0.920455,0.968434
350,0.209100,0.267480,0.890244,0.960526,0.829545,0.971867
400,0.201700,0.262854,0.893983,0.901734,0.886364,0.961490
450,0.169400,0.242703,0.906433,0.933735,0.880682,0.968000
500,0.168700,0.240618,0.917847,0.915254,0.920455,0.966225
